In [ ]:
import pandas as pd
from pyarrow import csv
import pyarrow.parquet as pq

###### Loading and formatting traffic data #########################################################

# Read the csv files into pandas DataFrames
years = ["2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014"]
weather = "weatherData.csv"
holidays = "Holidays.csv"

# Read the traffic data
traffic_data = pd.DataFrame()
for year in years:
    traffic_file = f"data/traffic-{year}.csv"
    traffic_df = pd.read_csv(traffic_file)
    traffic_data = pd.concat([traffic_data, traffic_df], ignore_index=True)

# convert dates to datetime
traffic_data['Dato'] = pd.to_datetime(traffic_data['Dato'], format='%Y-%m-%d')

# new dataframe with only Vej-ID and latlon
latlon_df = traffic_data[["Vej-Id", "latlon"]].drop_duplicates()
# convert to a dictionary mapping Vej-Id to latlon
latlon_dict = dict(zip(latlon_df["Vej-Id"], latlon_df["latlon"]))

times = ["kl.00-01", "kl.01-02", "kl.02-03", "kl.03-04", "kl.04-05", "kl.05-06",
         "kl.06-07", "kl.07-08", "kl.08-09", "kl.09-10", "kl.10-11", "kl.11-12",
         "kl.12-13", "kl.13-14", "kl.14-15", "kl.15-16", "kl.16-17", "kl.17-18",
         "kl.18-19", "kl.19-20", "kl.20-21", "kl.21-22", "kl.22-23", "kl.23-24"]
times2 = ["0", "1", "2", "3", "4", "5",
          "6", "7", "8", "9", "10", "11",
         "12", "13", "14", "15", "16", "17",
         "18", "19", "20", "21", "22", "23"]
tdict = dict(zip(times, times2))

# replace column names in traffic_data
for k, v in tdict.items():
    traffic_data.rename(columns={k: v}, inplace=True)

# First, melt so that each row represents a station-hour combo
traffic_long = traffic_data.melt(
    id_vars=['Vej-Id','Vejnavn','Dato','lat','lon','latlon'],
    value_vars=times2,
    var_name='hour',
    value_name='traffic_count'
)

# Ensure 'Dato' is datetime type
traffic_long['Dato'] = pd.to_datetime(traffic_long['Dato'])

# Convert hour string to integer, then to timedelta
traffic_long['hour'] = traffic_long['hour'].astype(int)
traffic_long['datetime'] = traffic_long['Dato'] + pd.to_timedelta(traffic_long['hour'], unit='h')

traffic_wide = traffic_long.pivot(index='datetime', columns='Vej-Id', values='traffic_count')
traffic_wide = traffic_wide.astype('Int64')  # capital 'I'

# Optional: sort by datetime
traffic_wide = traffic_wide.sort_index()

# fill NaN values with -1
traffic_wide = traffic_wide.fillna(-1)

# data usage of df
#print(traffic_wide.info(memory_usage='deep'))



####### Loading and adding weather data ###########################################################

# load weather data
weather_data = pd.read_csv("data/"+weather, sep=',',skiprows=3)
weather_data['time'] = pd.to_datetime(weather_data['time'])

# add wather data to traffic_wide, keep datetime as index
traffic_wide = traffic_wide.join(weather_data.set_index('time'), on='datetime', how='left')



###### Loading and adding holiday data ############################################################

# load holidays data
holidays_data = pd.read_csv("data/"+holidays, sep=',')

holidays_data['Date'] = pd.to_datetime(holidays_data['Date'], format='%d/%m/%Y')

# get date out of index and into column
traffic_wide["Date"] = traffic_wide.index

traffic_wide["DayOfWeek"] = traffic_wide["Date"].dt.day_name()
traffic_wide["isHoliday"] = traffic_wide["Date"].dt.date.isin(holidays_data["Date"].dt.date.values)
traffic_wide["isWeekend"] = traffic_wide["DayOfWeek"].isin(["Saturday", "Sunday"])
traffic_wide["isWeekday"] = ~traffic_wide["isWeekend"]

traffic_wide["holidayName"] = traffic_wide["Date"].dt.date.apply(
    lambda x: holidays_data.loc[holidays_data["Date"].dt.date == x, "HolidayName"].values[0] if x in holidays_data["Date"].dt.date.values else None
)
traffic_wide["holidayType"] = traffic_wide["Date"].dt.date.apply(
    lambda x: holidays_data.loc[holidays_data["Date"].dt.date == x, "HolidayType"].values[0] if x in holidays_data["Date"].dt.date.values else None
)

# drop the Date column
traffic_wide.drop(columns=["Date"], inplace=True)

###### How to use the data #######################################################################

# traffic_wide is the main dataframe, it contains the traffic data for all stations and all hours
# it is indexed by datetime and has columns for each station (Vej-Id)
# it also contains the weather data for each hour
# and the holiday data, non holiday days are marked as "isHoliday" = False and will have missing values for "holidayName" and "holidayType"
# the columns "isWeekend" and "isWeekday" are also added to indicate if the day is a weekend or a weekday

# since stations have their own columns (for each direction and total), location is stored separately
# to get location of a station, use the latlon_dict on column name (Vej-Id)


In [47]:
# save the dataframe to a parquet file
traffic_wide.to_parquet("data/traffic_data.parquet", engine='pyarrow', compression='snappy')

In [48]:
# save to csv file
traffic_wide.to_csv("data/traffic_data.csv", index=True, sep=',', encoding='utf-8')